### 수집할 데이터들
JOIN : 나라

T : 출산율

X : 인구, 영토크기, 인구밀도, GDP, 1인당GDP, GDP성장, 1인당GDP성장, 문해율, 교육수준

In [38]:
import requests #HTML 파일 불러오기
from bs4 import BeautifulSoup as bs #HTML 가공
import pandas as pd #DataFrame으로 예쁘게 변환
from IPython.display import display
import numpy as np
import re #정규식

In [14]:
# 나라랑 출산율 부터 크롤링
state = [] #나라 저장할 list
fertility = [] #출산율 저장할 list

url = r"https://en.wikipedia.org/wiki/List_of_countries_by_total_fertility_rate"
selector = r"#mw-content-text > div.mw-content-ltr.mw-parser-output > table:nth-child(35) > tbody > tr:nth-child(2) > td:nth-child(1) > table > tbody"

res = requests.get(url)
soup = bs(res.content)
table = soup.select_one(selector)

for tr in table.select("tr") : #표의 행들을 전부 가져오기
    td = tr.select("td") #표의 각 행 하나하나에서 열들을 전부 가져오기
    try :
        num = float(td[2].text) #원하는 값을 숫자로 바꿔주고
    except :
        continue #해당 값이 숫자가 아니면 행 넘어가기        

    try :
        float(td[0].text) #순위에 괄호가 있는 열 (속령) 넘기기
    except :
        continue
    
    state_name = td[1].text.strip() #strip 함수 : 앞뒤로 넣어진 공백문자들 제거
    state.append(state_name)
    fertility.append(num)

    if td[0].text == "235" :
        break

#list로 모은 자료들을 dict로 묶어주기

dic = {"state" : state, "fertility" : fertility,
      "population" : None, "territory" : None, "density" : None,
      "GDP" : None, "GDPcap" : None, "GDPgrowth" : None, "GDPcapgrowth" : None,
      "literacy" : None, "education" : None}

#dict를 DataFrame으로 변환

df = pd.DataFrame(dic)
display(df)


,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,None,None,None,None,None,None,None,None,None
1,Somalia,5.91,None,None,None,None,None,None,None,None,None
2,DR Congo,5.90,None,None,None,None,None,None,None,None,None
3,Central African Republic,5.81,None,None,None,None,None,None,None,None,None
4,Niger,5.79,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,None,None,None,None,None,None,None,None,None
196,Ukraine,1.00,None,None,None,None,None,None,None,None,None
197,Singapore,0.96,None,None,None,None,None,None,None,None,None
198,Taiwan,0.86,None,None,None,None,None,None,None,None,None


In [98]:
#DataFrame 채우는 작업 함수로 만들기

def fill_df(df,url,selector,col_name,join_name,col_idx,join_idx) :
    res = requests.get(url) #html 가져오기
    soup = bs(res.content) #beautifulsoup로 변환
    table = soup.select_one(selector) #table만 추출
    
    for tr in table.select("tr") : #테이블에서 모든 행 추출
        td = tr.select("td") #각 행에서 모든 열 추출
        try :
            num = float(re.sub(r"[^0-9.]","",string = td[col_idx].text)) #문자를 숫자로, 정규식에서 부정 연산자는 ^
        except :
            continue
        # 인구 수 넣을 때 나라 이름에 맞춰서 넣어줘야 함
    
        state = re.sub(r"[^a-zA-Z가-힣\s]","",string = td[join_idx].text).strip() #나라이름 가져오고, 정규식에서 띄어쓰기(공백문자, 스페이스)는 \s
        for i in range(len(df)) :
            if df.loc[i, join_name] == state :
                df.loc[i, col_name] = num #나라이름 찾으면 해당 값 넣어주기
                break
    
    return df    

In [22]:
# population 채우기

url = r"https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"
selector = r"#mw-content-text > div.mw-content-ltr.mw-parser-output > table > tbody"
col_name = "population"
join_name = "state"
col_idx = 1
join_idx = 0

df = fill_df(df,url,selector,col_name,join_name,col_idx,join_idx)

display(df)


,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,18675547.0,None,None,None,None,None,None,None,None
1,Somalia,5.91,19009151.0,None,None,None,None,None,None,None,None
2,DR Congo,5.90,None,None,None,None,None,None,None,None,None
3,Central African Republic,5.81,6470307.0,None,None,None,None,None,None,None,None
4,Niger,5.79,26312034.0,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,882.0,None,None,None,None,None,None,None,None
196,Ukraine,1.00,32962000.0,None,None,None,None,None,None,None,None
197,Singapore,0.96,6036900.0,None,None,None,None,None,None,None,None
198,Taiwan,0.86,23374742.0,None,None,None,None,None,None,None,None


In [24]:
# territory 채우기

url = r"https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density"
selector = r"#mw-content-text > div.mw-content-ltr.mw-parser-output > table > tbody"
col_name = "territory"
join_name = "state"
col_idx = 4
join_idx = 0

df = fill_df(df,url,selector,col_name,join_name,col_idx,join_idx)

display(df)


,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,18675547.0,1259200.0,None,None,None,None,None,None,None
1,Somalia,5.91,19009151.0,627340.0,None,None,None,None,None,None,None
2,DR Congo,5.90,None,2267050.0,None,None,None,None,None,None,None
3,Central African Republic,5.81,6470307.0,622980.0,None,None,None,None,None,None,None
4,Niger,5.79,26312034.0,1266700.0,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,882.0,0.5,None,None,None,None,None,None,None
196,Ukraine,1.00,32962000.0,579400.0,None,None,None,None,None,None,None
197,Singapore,0.96,6036900.0,718.0,None,None,None,None,None,None,None
198,Taiwan,0.86,23374742.0,35410.0,None,None,None,None,None,None,None


In [26]:
# density 채우기

url = r"https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density"
selector = r"#mw-content-text > div.mw-content-ltr.mw-parser-output > table > tbody"
col_name = "density"
join_name = "state"
col_idx = 1
join_idx = 0

df = fill_df(df,url,selector,col_name,join_name,col_idx,join_idx)

display(df)

,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,18675547.0,1259200.0,16.0,None,None,None,None,None,None
1,Somalia,5.91,19009151.0,627340.0,30.0,None,None,None,None,None,None
2,DR Congo,5.90,None,2267050.0,48.0,None,None,None,None,None,None
3,Central African Republic,5.81,6470307.0,622980.0,8.6,None,None,None,None,None,None
4,Niger,5.79,26312034.0,1266700.0,21.0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,882.0,0.5,1800.0,None,None,None,None,None,None
196,Ukraine,1.00,32962000.0,579400.0,65.0,None,None,None,None,None,None
197,Singapore,0.96,6036900.0,718.0,8120.0,None,None,None,None,None,None
198,Taiwan,0.86,23374742.0,35410.0,656.0,None,None,None,None,None,None


In [28]:
#GDP

url = r"https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
selector = r"#mw-content-text > div.mw-content-ltr.mw-parser-output > table.wikitable.sortable.sticky-header-multi > tbody"
col_name = "GDP"
join_name = "state"
col_idx = 1
join_idx = 0

df = fill_df(df,url, selector,col_name,join_name, col_idx, join_idx)

display(df)

,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,18675547.0,1259200.0,16.0,18792.0,None,None,None,None,None
1,Somalia,5.91,19009151.0,627340.0,30.0,12994.0,None,None,None,None,None
2,DR Congo,5.90,None,2267050.0,48.0,79119.0,None,None,None,None,None
3,Central African Republic,5.81,6470307.0,622980.0,8.6,2932.0,None,None,None,None,None
4,Niger,5.79,26312034.0,1266700.0,21.0,21874.0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,882.0,0.5,1800.0,None,None,None,None,None,None
196,Ukraine,1.00,32962000.0,579400.0,65.0,205742.0,None,None,None,None,None
197,Singapore,0.96,6036900.0,718.0,8120.0,564774.0,None,None,None,None,None
198,Taiwan,0.86,23374742.0,35410.0,656.0,804889.0,None,None,None,None,None


In [30]:
#GDPperCaP

url = r"https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita"
selector = r"#mw-content-text > div.mw-content-ltr.mw-parser-output > table > tbody"
col_name = "GDPcap"
join_name = "state"
col_idx = 1
join_idx = 0

df = fill_df(df,url, selector,col_name,join_name, col_idx, join_idx)

display(df)

,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,18675547.0,1259200.0,16.0,18792.0,991.0,None,None,None,None
1,Somalia,5.91,19009151.0,627340.0,30.0,12994.0,766.0,None,None,None,None
2,DR Congo,5.90,None,2267050.0,48.0,79119.0,743.0,None,None,None,None
3,Central African Republic,5.81,6470307.0,622980.0,8.6,2932.0,532.0,None,None,None,None
4,Niger,5.79,26312034.0,1266700.0,21.0,21874.0,751.0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,882.0,0.5,1800.0,None,None,None,None,None,None
196,Ukraine,1.00,32962000.0,579400.0,65.0,205742.0,6261.0,None,None,None,None
197,Singapore,0.96,6036900.0,718.0,8120.0,564774.0,92932.0,None,None,None,None
198,Taiwan,0.86,23374742.0,35410.0,656.0,804889.0,34426.0,None,None,None,None


In [32]:
#GDPgrowthy

url = r"https://en.wikipedia.org/wiki/List_of_countries_by_real_GDP_growth_rate"
selector = r"#mw-content-text > div.mw-content-ltr.mw-parser-output > table:nth-child(11) > tbody"
col_name = "GDPgrowth"
join_name = "state"
col_idx = 1
join_idx = 0

df = fill_df(df,url, selector,col_name,join_name, col_idx, join_idx)

display(df)

,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,18675547.0,1259200.0,16.0,18792.0,991.0,3.2,None,None,None
1,Somalia,5.91,19009151.0,627340.0,30.0,12994.0,766.0,4.0,None,None,None
2,DR Congo,5.90,None,2267050.0,48.0,79119.0,743.0,4.7,None,None,None
3,Central African Republic,5.81,6470307.0,622980.0,8.6,2932.0,532.0,1.4,None,None,None
4,Niger,5.79,26312034.0,1266700.0,21.0,21874.0,751.0,9.9,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,882.0,0.5,1800.0,None,None,None,None,None,None
196,Ukraine,1.00,32962000.0,579400.0,65.0,205742.0,6261.0,3.0,None,None,None
197,Singapore,0.96,6036900.0,718.0,8120.0,564774.0,92932.0,2.6,None,None,None
198,Taiwan,0.86,23374742.0,35410.0,656.0,804889.0,34426.0,3.7,None,None,None


In [34]:
#GDPcapgrowth

url = r"https://en.wikipedia.org/wiki/List_of_countries_by_real_GDP_per_capita_growth"
selector = r"#mw-content-text > div.mw-content-ltr.mw-parser-output > table > tbody"
col_name = "GDPcapgrowth"
join_name = "state"
col_idx = 1
join_idx = 0

df = fill_df(df,url, selector,col_name,join_name, col_idx, join_idx)

display(df)

,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,18675547.0,1259200.0,16.0,18792.0,991.0,3.2,1.0,None,None
1,Somalia,5.91,19009151.0,627340.0,30.0,12994.0,766.0,4.0,0.0,None,None
2,DR Congo,5.90,None,2267050.0,48.0,79119.0,743.0,4.7,5.1,None,None
3,Central African Republic,5.81,6470307.0,622980.0,8.6,2932.0,532.0,1.4,-2.0,None,None
4,Niger,5.79,26312034.0,1266700.0,21.0,21874.0,751.0,9.9,-1.2,None,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,882.0,0.5,1800.0,None,None,None,None,None,None
196,Ukraine,1.00,32962000.0,579400.0,65.0,205742.0,6261.0,3.0,-17.1,None,None
197,Singapore,0.96,6036900.0,718.0,8120.0,564774.0,92932.0,2.6,-3.7,None,None
198,Taiwan,0.86,23374742.0,35410.0,656.0,804889.0,34426.0,3.7,None,None,None


In [96]:
#literacy

url = r"https://en.wikipedia.org/wiki/List_of_countries_by_literacy_rate"
selector = r"#mw-content-text > div.mw-content-ltr.mw-parser-output > table:nth-child(15) > tbody"
col_name = "literacy"
join_name = "state"
col_idx = 1
join_idx = 0

df = fill_df(df,url, selector,col_name,join_name, col_idx, join_idx)

display(df)

,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,18675547.0,1259200.0,16.0,18792.0,991.0,3.2,1.0,22.3,None
1,Somalia,5.91,19009151.0,627340.0,30.0,12994.0,766.0,4.0,0.0,39.0,None
2,DR Congo,5.90,None,2267050.0,48.0,79119.0,743.0,4.7,5.1,77.0,None
3,Central African Republic,5.81,6470307.0,622980.0,8.6,2932.0,532.0,1.4,-2.0,37.4,None
4,Niger,5.79,26312034.0,1266700.0,21.0,21874.0,751.0,9.9,-1.2,35.1,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,882.0,0.5,1800.0,None,None,None,None,None,None
196,Ukraine,1.00,32962000.0,579400.0,65.0,205742.0,6261.0,3.0,-17.1,99.8,None
197,Singapore,0.96,6036900.0,718.0,8120.0,564774.0,92932.0,2.6,-3.7,97.5,None
198,Taiwan,0.86,23374742.0,35410.0,656.0,804889.0,34426.0,3.7,None,98.7,None


In [136]:
#education

url = r"https://en.wikipedia.org/wiki/Education_Index"
selector = r"table.wikitable.sortable"
res = requests.get(url) #html 가져오기
soup = bs(res.content) #html 가공을 위해 BeautifulSoup로 변환
table = soup.select_one(selector) #html에서 테이블만 가져옴

for tr in table.select("tr") : #table의 행(tr) 가져오기
    td = tr.select("td") #각 tr에서 열(td) 가져오기
    try :
        num = float(re.sub(r"[^0-9.]","",string=td[-1].text))
    except :
        continue
    state_name1 = re.sub(r"^a-zA-z가-힣","",string=td[0].select_one("a").get('title')).strip() #title 속성을 추출
    state_name2 = re.sub(r"^a-zA-z가-힣","",string=td[0].text).strip() #텍스트 추출

    for i in range(len(df)) :
        if (df.loc[i, "state"] == state_name1) or (df.loc[i, "state"] == state_name2) :
            df.loc[i, "education"] = num
            break

display(df)

,state,fertility,population,territory,density,GDP,GDPcap,GDPgrowth,GDPcapgrowth,literacy,education
0,Chad,5.94,18675547.0,1259200.0,16.0,18792.0,991.0,3.2,1.0,22.3,0.288
1,Somalia,5.91,19009151.0,627340.0,30.0,12994.0,766.0,4.0,0.0,39.0,None
2,DR Congo,5.90,None,2267050.0,48.0,79119.0,743.0,4.7,5.1,77.0,None
3,Central African Republic,5.81,6470307.0,622980.0,8.6,2932.0,532.0,1.4,-2.0,37.4,0.353
4,Niger,5.79,26312034.0,1266700.0,21.0,21874.0,751.0,9.9,-1.2,35.1,0.249
...,...,...,...,...,...,...,...,...,...,...,...
195,Vatican City,1.00,882.0,0.5,1800.0,None,None,None,None,None,None
196,Ukraine,1.00,32962000.0,579400.0,65.0,205742.0,6261.0,3.0,-17.1,99.8,0.799
197,Singapore,0.96,6036900.0,718.0,8120.0,564774.0,92932.0,2.6,-3.7,97.5,0.924
198,Taiwan,0.86,23374742.0,35410.0,656.0,804889.0,34426.0,3.7,None,98.7,None


In [138]:
#DataFrame 저장

df.to_csv("fertility_data.csv")

In [4]:
# r의 기능

print(r"hello\tworld")
print("hello\tworld")

hello\tworld
hello	world


In [6]:
#strip() 의 기능

text = "\t\thello world\t\t"

print(text)
print(text.strip())

		hello world		
hello world


In [ ]:
#get 함수 -> 속성을 추출